In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.vgg19 import VGG19
from keras.applications.imagenet_utils import preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortex"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = VGG19(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 512)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 512)

In [ ]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [ ]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.9min


[CV] ........................... C=0.01, score=0.924510, total= 1.9min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.930417, total= 1.9min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.938492, total= 1.9min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.945491, total= 1.9min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.920870, total= 1.9min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.943898, total= 1.9min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.916016, total= 2.0min
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  2.2min


[CV] ............................ C=0.1, score=0.932087, total= 2.1min
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.877953, total= 2.1min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.911133, total= 2.1min
[CV] ............................ C=1.0, score=0.909804, total= 2.1min
[CV] ............................ C=0.1, score=0.906496, total= 2.1min
[CV] C=10.0 ..........................................................
[CV] C=100.0 .........................................................
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.928571, total= 2.1min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.928571, total= 2.2min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] .......................... C=100.0, score=0.909180, total= 2.1min
[CV] C=10000.0 .......................................................
[CV] ........................... C=10.0, score=0.930624, total= 2.1min
[CV] C=10000.0 .......................................................
[CV] .......................... C=100.0, score=0.917323, total= 2.1min
[CV] C=10000.0 .......................................................
[CV] ........................... C=10.0, score=0.922619, total= 2.2min
[CV] C=10000.0 .......................................................
[CV] .......................... C=100.0, score=0.921722, total= 2.2min
[CV] C=10000.0 .......................................................
[CV] .......................... C=100.0, score=0.925490, total= 2.2min
[CV] C=10000.0 .......................................................
[CV] .......................... C=100.0, score=0.897638, total= 2.2min
[CV] C=10000.0 .......................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:  6.2min


[CV] ......................... C=1000.0, score=0.919608, total= 2.1min
[CV] C=100000.0 ......................................................
[CV] ......................... C=1000.0, score=0.879921, total= 2.2min
[CV] C=100000.0 ......................................................
[CV] ......................... C=1000.0, score=0.912916, total= 2.2min
[CV] C=100000.0 ......................................................
[CV] ......................... C=1000.0, score=0.928150, total= 2.2min
[CV] C=100000.0 ......................................................
[CV] ......................... C=1000.0, score=0.887897, total= 2.1min
[CV] C=100000.0 ......................................................
[CV] ......................... C=1000.0, score=0.924303, total= 2.1min
[CV] C=1000000.0 .....................................................
[CV] ........................ C=10000.0, score=0.924510, total= 2.1min
[CV] C=1000000.0 .....................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:  6.6min


[CV] ........................ C=10000.0, score=0.894841, total= 2.2min
[CV] C=1000000.0 .....................................................
[CV] ........................ C=10000.0, score=0.908012, total= 2.2min
[CV] C=10000000.0 ....................................................
[CV] ........................ C=10000.0, score=0.922696, total= 2.2min
[CV] C=10000000.0 ....................................................
[CV] ........................ C=10000.0, score=0.923459, total= 2.2min
[CV] C=10000000.0 ....................................................
[CV] ....................... C=100000.0, score=0.911133, total= 2.1min
[CV] C=10000000.0 ....................................................
[CV] ........................ C=10000.0, score=0.914343, total= 2.3min
[CV] C=10000000.0 ....................................................
[CV] ....................... C=100000.0, score=0.912916, total= 2.2min
[CV] C=10000000.0 ....................................................


In [15]:
top_model.cv_results_

{'mean_fit_time': array([115.3173564 , 129.08191085, 128.73255148, 129.56497047,
        130.10511367, 129.4792789 , 129.71932068, 127.35410295,
        121.47013018, 109.24742823,  77.60421741,  85.39110956,
         53.00868065]),
 'mean_score_time': array([0.03301179, 0.03239067, 0.02518015, 0.02128386, 0.01886697,
        0.02005999, 0.01957543, 0.01820817, 0.01483808, 0.01480372,
        0.01141346, 0.00817463, 0.00559726]),
 'mean_test_score': array([0.92926204, 0.92058011, 0.91525257, 0.91643646, 0.91692976,
        0.90943173, 0.9120955 , 0.90943173, 0.91436464, 0.91475927,
        0.91268745, 0.91308208, 0.90765588]),
 'mean_train_score': array([0.97523691, 0.97752866, 0.97381103, 0.97528203, 0.97751848,
        0.96778137, 0.96931972, 0.96869306, 0.9743695 , 0.97492836,
        0.9737033 , 0.97470001, 0.96353238]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [16]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.92926, std: 0.01177, params: {'C': 0.01},
 mean: 0.92058, std: 0.01172, params: {'C': 0.1},
 mean: 0.91525, std: 0.01540, params: {'C': 1.0},
 mean: 0.91644, std: 0.01275, params: {'C': 10.0},
 mean: 0.91693, std: 0.01116, params: {'C': 100.0},
 mean: 0.90943, std: 0.01683, params: {'C': 1000.0},
 mean: 0.91210, std: 0.01429, params: {'C': 10000.0},
 mean: 0.90943, std: 0.01217, params: {'C': 100000.0},
 mean: 0.91436, std: 0.01449, params: {'C': 1000000.0},
 mean: 0.91476, std: 0.01431, params: {'C': 10000000.0},
 mean: 0.91269, std: 0.01106, params: {'C': 100000000.0},
 mean: 0.91308, std: 0.01119, params: {'C': 1000000000.0},
 mean: 0.90766, std: 0.01948, params: {'C': 10000000000.0}]

In [17]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.01} - score: 0.9293
